# **Import Library**

In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

# **Upload Data**

In [3]:
df =pd.read_csv('/content/covid_toy.csv')

In [4]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [5]:
df['cough'].value_counts()

Mild      62
Strong    38
Name: cough, dtype: int64

In [6]:
df['city'].value_counts()

Kolkata      32
Bangalore    30
Delhi        22
Mumbai       16
Name: city, dtype: int64

# **Check Missing Value**

In [7]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [8]:
df['fever'].shape

(100,)

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test , y_train, y_test =train_test_split(df.drop(columns=['has_covid']), df['has_covid'],
                                                    test_size=0.2)

In [10]:
X_train

,age,gender,fever,cough,city
88,5,Female,100.0,Mild,Kolkata
4,65,Female,101.0,Mild,Mumbai
72,83,Female,101.0,Mild,Kolkata
26,19,Female,100.0,Mild,Kolkata
99,10,Female,98.0,Strong,Kolkata
...,...,...,...,...,...
57,49,Female,99.0,Strong,Bangalore
98,5,Female,98.0,Strong,Mumbai
41,82,Male,NaN,Mild,Kolkata
77,8,Female,101.0,Mild,Kolkata


In [11]:
X_test

,age,gender,fever,cough,city
38,49,Female,101.0,Mild,Delhi
56,71,Male,NaN,Strong,Kolkata
69,73,Female,103.0,Mild,Delhi
8,19,Female,100.0,Strong,Bangalore
78,11,Male,100.0,Mild,Bangalore
19,42,Female,NaN,Strong,Bangalore
87,47,Male,101.0,Strong,Bangalore
23,80,Female,98.0,Mild,Delhi
54,60,Female,99.0,Mild,Mumbai
85,16,Female,103.0,Mild,Bangalore


In [12]:
y_train

88     No
4      No
72     No
26    Yes
99    Yes
     ... 
57     No
98     No
41    Yes
77     No
25     No
Name: has_covid, Length: 80, dtype: object

# **Apply separate columns**

In [19]:
#adding simple imputer to fever col
si =SimpleImputer()
X_train_fever =si.fit_transform(X_train[['fever']])

#also the test data
X_test_fever = si.transform(X_test[['fever']])

In [20]:
train =X_train_fever.shape
test =X_test_fever.shape

print("X Train Fever Shpe", train)
print("X Test Fever shape ", test)

X Train Fever Shpe (80, 1)
X Test Fever shape  (20, 1)


In [22]:
#ordinal encoding - > cough

oe =OrdinalEncoder(categories=[['Mild', 'Strong']] )
X_train_cough = oe.fit_transform(X_train[['cough']])

#also the test data
X_test_cough =oe.transform(X_test[['cough']])


In [23]:
cough_train =X_train_cough.shape
cough_test =X_test_cough.shape

print("X Train shape in cough columns" ,cough_train)
print("X test shape in cough columns ", cough_test)

X Train shape in cough columns (80, 1)
X test shape in cough columns  (20, 1)


In [24]:
ohe = OneHotEncoder (drop='first', sparse=False)
X_train_gender_city = ohe.fit_transform(X_train[['gender', 'city']])

#also the test data
X_test_gender_city  = ohe.transform(X_test[['gender', 'city']])


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [25]:
Train =X_train_gender_city.shape
Test = X_test_gender_city.shape

print("X train geder sahpe", Train)
print("X Test gender shape", Test)

X train geder sahpe (80, 4)
X Test gender shape (20, 4)


# **Exact the Age columns**

In [26]:
# Extracting Age
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values

# also the test data
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values

X_train_age.shape

(80, 1)

In [27]:
#Extracting Age
X_train_age =X_train.drop(columns=['gender', 'fever', 'cough', 'city']).values

#also the test data
X_test_age =X_test.drop(columns=['gender', 'fever', 'cough', 'city']).values

X_train_age.shape

(80, 1)

# **concatenate Mean (Combine Data)**

In [28]:
X_train_transformed = np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough),axis=1)
# also the test data
X_test_transformed = np.concatenate((X_test_age,X_test_fever,X_test_gender_city,X_test_cough),axis=1)

X_train_transformed.shape

(80, 7)

# **Columns Transform**

This column transformer applies the scaler to the numerical features and the encoder to the categorical features. The preprocessor object can then be used to preprocess the data before training a machine learning model.

In [29]:
from sklearn.compose import ColumnTransformer

In [30]:
transformer =ColumnTransformer(transformers=[
    ('tnf1', SimpleImputer(), ['fever']),
    ('tnf2', OrdinalEncoder(categories=[['Mild', 'Strong']]),['cough'] ),
    ('tnf3', OneHotEncoder(sparse=False,drop='first'), ['gender', 'city'])

], remainder = 'passthrough')

# **Shape the Columns**

In [31]:
Columns_train =transformer.fit_transform(X_train).shape

Columns_test= transformer.transform(X_test).shape

print("Train the columns Shape", Columns_train)
print("Test the Columns Shaoe", Columns_test)

Train the columns Shape (80, 7)
Test the Columns Shaoe (20, 7)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
